In [1]:
import numpy as np
import random
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()

### Single Layer, No Batches

In [2]:
inputs = np.array([1, 2, 3, 2.5])

weights = np.array([[0.2, 0.8, -0.5, 1.0],
                    [0.5, -0.91, 0.26, -0.5],
                    [-0.26, -0.27, 0.17, 0.87]])

weights = np.array([[0.2, 0.5, -0.26],
                    [0.8, -0.91, -0.27],
                    [-0.5, -0.26, 0.17],
                    [1.0, -0.5, 0.87]])

biases = np.array([2, 3, 0.5])

output = np.dot(inputs, weights) + biases
print(output)

[ 4.79999995 -0.3499999   2.38499999]


### Single Layer, With Batches

In [3]:
inputs = np.array([[1, 2, 3, 2.5],
                   [2.0, 5.0, -1.0, 2.0],
                   [-1.5, 2.7, 3.3, -0.8]])

weights = np.array([[0.2, 0.5, -0.26],
                    [0.8, -0.91, -0.27],
                    [-0.5, -0.26, 0.17],
                    [1.0, -0.5, 0.87]])

biases = np.array([2, 3, 0.5])

output = np.dot(inputs, weights) + biases
print(output)

[[ 4.79999995 -0.3499999   2.38499999]
 [ 8.9000001  -1.28999996  0.19999999]
 [ 1.41000003 -0.66499996  0.02599999]]


### Dense Layer Class & Activation Functions

Let $X$ be in the input matrix to the layer such that, 

$X = 
\begin{bmatrix}
    X_{11} & X_{12} & X_{13} & \dots  & X_{1M} \\
    X_{21} & X_{22} & X_{23} & \dots  & X_{2M} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    X_{S1} & X_{S2} & X_{S3} & \dots  & X_{SM}
\end{bmatrix}$

In which, $S$ is the sample size and $M$ is the number of inputs to a given neuron. The weights of the layer $W$ are then, 

$W = 
\begin{bmatrix}
    W_{11} & W_{12} & W_{13} & \dots  & W_{1N} \\
    W_{21} & W_{22} & W_{23} & \dots  & W_{2N} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    W_{M1} & W_{M2} & W_{M3} & \dots  & W_{MN}
\end{bmatrix}$

Likewise, we have the bias $B$ as, 

$B = 
\begin{bmatrix}
    B_{1} & B_{2} & B_{3} & \dots  & B_{4} \\
\end{bmatrix}$

The output of the layer $Y$ is then given via $\sigma(XW + B) = Y$ where $\sigma(z)$ is the activation function of choice.

In [4]:
class Activation_ReLU:
    """The rectified linear function."""
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_SoftMAX:
    """The normalised expontential function or 'softmax function'."""
    def forward(self, inputs):

        # Overflow prevention,
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))
        
        self.output = exp_values/np.sum(exp_values, axis = 1, keepdims = True)

class Layer_Dense:
    """
    Parameters:
    inputs - Number of neurons in the layer.
    neurons - Number of inputs for a given neuron in the later.
    batch_size - The batch size.
    """

    def __init__(self, inputs, neurons):
        """The weights are intially set to a random value between
        -1 and 1."""
        self.weights = 0.1*np.random.randn(inputs, neurons)
        self.biases = np.zeros((1, neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

X, y = spiral_data(samples = 5, classes = 3)

# Creating neutral network,
LAYER1 = Layer_Dense(inputs = 2, neurons = 3)
LAYER2 = Layer_Dense(inputs = 3, neurons = 3)
ACTIVATION1 = Activation_ReLU()
ACTIVATION2 = Activation_SoftMAX()

# Feed-Forward Propagation,
LAYER1.forward(inputs = X)
ACTIVATION1.forward(LAYER1.output)
LAYER2.forward(inputs = ACTIVATION1.output)
ACTIVATION2.forward(LAYER2.output)
print(ACTIVATION2.output)

[[0.33333334 0.33333334 0.33333334]
 [0.33358076 0.3323869  0.33403233]
 [0.33333334 0.33333334 0.33333334]
 [0.33706257 0.32853162 0.33440587]
 [0.331975   0.3340509  0.3339741 ]
 [0.33333334 0.33333334 0.33333334]
 [0.3334715  0.3335453  0.33298326]
 [0.3357063  0.33018556 0.33410814]
 [0.33333334 0.33333334 0.33333334]
 [0.3423476  0.3267132  0.3309392 ]
 [0.33333334 0.33333334 0.33333334]
 [0.3323655  0.33384457 0.3337899 ]
 [0.3335868  0.3337223  0.33269086]
 [0.33632594 0.328948   0.33472607]
 [0.33333334 0.33333334 0.33333334]]


### The Loss Function